# Libs

In [2]:
import pandas as pd
import sys, os 
sys.path.append(os.path.abspath(".."))

from scr.utils import (
    normalizar_pais,
    agrupar_pais,
    comprobar_normalidad,
    correlacion_variables,
    comprobar_nomalidad_y_varianza,
    calcular_fuerza_asociacion_2_grupos,
    calcular_fuerza_asociacion_mas_2_grupos,
    cramers_v,
    calcular_independencia_categoricas,
    agrupar_tramo,
    cohens_d,
    r_mannwhitney,
    prueba_comparacion_2grupos_variable
)

# Load

In [20]:
df_convo_deuda = pd.read_csv('../data/processed/data.csv')

# EDA

## Vista preliminar

In [21]:
df_convo_deuda.head()

,DIA CERO,FECHA PRODUCCIÓN,ID PROGRAMA,PRECIO CURSO,FACTURACIÓN NETA,IMPORTE INSCRIPCIÓN,% INSCRIPCION,MEDIO PAGO PI,IMPORTE PENDIENTE PAGO,MEDIO PAGO IMPORTE PENDIENTE,...,PAÍS DE RESIDENCIA,AGRUPACION NACIONALIDAD,AGRUPACION PAÍS DE RESIDENCIA,MANTIENE MEDIO PAGO,DIFERENCIA FECHA 1ra CUOTA - ORIGINAL,DIFERENCIA FECHA PRODUCCIÓN - 1ra CUOTA,% DTO,Moroso,% Impagado Actual Vdo,Importe Impagado Actual
0,False,2023-05-23,240433990,13900.0,7993.89,799.89,0.100063,TARJETA,7194.00,LINK DE PAGO,...,Chile,CHILE,CHILE,False,0.0,39.0,0.424900,False,0.000000,0.0
1,False,2023-05-31,240433720,7500.0,4937.62,740.64,0.149999,TARJETA,4196.98,LINK DE PAGO,...,Ecuador,ECUADOR,ECUADOR,False,0.0,184.0,0.341651,True,0.850008,4197.0
2,False,2023-05-31,240433150,7500.0,5343.75,801.56,0.150000,TARJETA,4542.19,TARJETA,...,Perú,PERÚ,PERÚ,True,0.0,306.0,0.287500,False,0.000000,0.0
3,False,2023-07-03,240433400,7500.0,4809.37,721.41,0.150001,TARJETA,4087.96,TARJETA,...,Nicaragua,R. LATAM,R. LATAM,True,0.0,90.0,0.358751,False,0.000000,0.0
4,False,2023-07-17,240433700,7500.0,5062.50,506.25,0.100000,PAYBAY,4556.25,LINK DE PAGO,...,México,MÉXICO,MÉXICO,False,0.0,46.0,0.325000,False,0.000000,0.0


In [14]:
df_convo_deuda.columns

Index(['TIPOLOGIA MATRICULA', 'TIPOLOGIA ALUMNO', 'ID OPORTUNIDAD', 'ID NACS',
       'DIA CERO', 'FECHA PRODUCCIÓN', 'CONVOCATORIA', 'ID PROGRAMA',
       'PRECIO CURSO', 'FACTURACIÓN NETA', 'IMPORTE INSCRIPCIÓN',
       '% INSCRIPCION', 'MEDIO PAGO PI', 'IMPORTE PENDIENTE PAGO',
       'MEDIO PAGO IMPORTE PENDIENTE', 'NUMERO DE CUOTAS', 'FECHA 1ra CUOTA',
       'FORMA DE PAGO', 'ASESOR', 'FECHA 1ra CUOTA ORIGINAL',
       'FORMA DE PAGO ORIGINAL', 'TIPO PROGRAMA', 'NACIONALIDAD',
       'PAÍS DE RESIDENCIA', 'AGRUPACION NACIONALIDAD',
       'AGRUPACION PAÍS DE RESIDENCIA', 'MANTIENE MEDIO PAGO',
       'DIFERENCIA FECHA 1ra CUOTA - ORIGINAL',
       'DIFERENCIA FECHA PRODUCCIÓN - 1ra CUOTA', '% DTO', 'Moroso',
       '% Impagado Actual Vdo', 'Importe Impagado Actual'],
      dtype='object')

In [13]:
df_convo_deuda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2595 entries, 0 to 2594
Data columns (total 33 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   TIPOLOGIA MATRICULA                      2595 non-null   object 
 1   TIPOLOGIA ALUMNO                         2595 non-null   object 
 2   ID OPORTUNIDAD                           2595 non-null   object 
 3   ID NACS                                  2595 non-null   int64  
 4   DIA CERO                                 2595 non-null   bool   
 5   FECHA PRODUCCIÓN                         2595 non-null   object 
 6   CONVOCATORIA                             2595 non-null   float64
 7   ID PROGRAMA                              2595 non-null   int64  
 8   PRECIO CURSO                             2595 non-null   float64
 9   FACTURACIÓN NETA                         2595 non-null   float64
 10  IMPORTE INSCRIPCIÓN                      2595 no

In [15]:
df_convo_deuda.describe()

,ID NACS,CONVOCATORIA,ID PROGRAMA,PRECIO CURSO,FACTURACIÓN NETA,IMPORTE INSCRIPCIÓN,% INSCRIPCION,IMPORTE PENDIENTE PAGO,NUMERO DE CUOTAS,DIFERENCIA FECHA 1ra CUOTA - ORIGINAL,DIFERENCIA FECHA PRODUCCIÓN - 1ra CUOTA,% DTO,% Impagado Actual Vdo,Importe Impagado Actual
count,2.595000e+03,2595.000000,2.595000e+03,2595.000000,2595.00000,2595.000000,2595.000000,2.595000e+03,2595.000000,2595.000000,2595.000000,2595.000000,2595.000000,2595.000000
mean,1.306802e+06,2405.484393,2.405821e+08,8201.811175,5413.66044,1002.512316,0.188317,4.411148e+03,10.244316,1.695568,62.253565,0.336374,0.088194,450.737495
std,1.173358e+05,2.589502,2.589603e+05,1903.907993,1141.64400,831.012827,0.154078,1.316023e+03,5.569879,15.093151,68.773681,0.052016,0.228454,1245.230764
min,2.020200e+04,2404.000000,2.404331e+08,7500.000000,1749.75000,0.000000,0.000000,-9.094947e-13,0.000000,-122.000000,-316.000000,0.000000,0.000000,0.000000
25%,1.313448e+06,2404.000000,2.404336e+08,7500.000000,4809.37000,730.770000,0.149999,4.052695e+03,5.500000,0.000000,20.000000,0.298900,0.000000,0.000000
50%,1.319514e+06,2404.000000,2.404338e+08,7500.000000,5076.56000,800.990000,0.150000,4.285910e+03,12.000000,0.000000,35.000000,0.335000,0.000000,0.000000
75%,1.325901e+06,2404.000000,2.404340e+08,7700.000000,5347.33000,1000.425000,0.199999,4.542190e+03,14.000000,0.000000,90.000000,0.374568,0.000000,0.000000
max,1.335944e+06,2410.000000,2.410340e+08,13900.000000,13900.00000,9771.700000,1.000000,1.181500e+04,21.000000,214.000000,1140.000000,0.766700,1.000000,11997.300000


In [ ]:
#Número de observación de cada categoría de la variable de interés.
df_convo_deuda['Moroso'].value_counts()

Moroso
False    2104
True      491
Name: count, dtype: int64

## Relación entre el % de Pago inicial (PI) y el importe imagado vencido

In [ ]:
## que guarde directamente el grafico en la carpeta de reports como 01_xxxx.jpg